###  Building a tokenizer, block by block

As we’ve seen in the previous sections, tokenization comprises several steps:

- Normalization (any cleanup of the text that is deemed necessary, such as removing spaces or accents, Unicode normalization, etc.)

- Pre-tokenization (splitting the input into words)

- Running the input through the model (using the pre-tokenized words to produce a sequence of tokens)

- Post-processing (adding the special tokens of the tokenizer, generating the attention mask and token type IDs)

As a reminder, here’s another look at the overall process:

![process](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter6/tokenization_pipeline-dark.svg)